* Data from https://opendata.dwd.de/climate_environment/CDC/grids_europe/monthly/cloud_cover/SEVIRI/

In [1]:
using YAXArrays,NetCDF

In [2]:
data=open_dataset("CFCmm202302010000410UDSVMSG01UD.nc")

YAXArray Dataset
Shared Axes: 
Dim{:lon} Sampled{Float64} -59.75:0.5:59.75 ForwardOrdered Regular Points,
Dim{:lat} Sampled{Float64} 79.75:-0.5:30.25 ReverseOrdered Regular Points,
Ti Sampled{DateTime} DateTime[DateTime("2023-02-01T00:00:00")] ForwardOrdered Irregular Points
Variables: 
nobs, cfc_std, cfc_day, nobs_day, cfc_middle, cfc, nobs_night, cfc_high, cfc_night, cfc_low, 
Properties: Dict{String, Any}("date_created" => "2023-03-01T07:50:53Z", "geospatial_lon_units" => "degrees_east", "instrument_vocabulary" => "GCMD Instruments, Version 8.1", "geospatial_lon_min" => -90.0f0, "creator_email" => "contact.cmsaf@dwd.de", "keywords" => "EARTH SCIENCE > ATMOSPHERE > CLOUDS > CLOUD PROPERTIES > CLOUD FRACTION", "time_coverage_end" => "2023-02-28T23:59:59Z", "time_coverage_resolution" => "P1M", "geospatial_lat_resolution" => "0.05 degree", "time_coverage_start" => "2023-02-01T00:00:00Z"…)


In [3]:
cfc_day=data["cfc_day"]
cfc_night=data["cfc_night"]

240×100×1 YAXArray{Union{Missing, Float32},3} with dimensions: 
  Dim{:lon} Sampled{Float64} -59.75:0.5:59.75 ForwardOrdered Regular Points,
  Dim{:lat} Sampled{Float64} 79.75:-0.5:30.25 ReverseOrdered Regular Points,
  Ti Sampled{DateTime} DateTime[DateTime("2023-02-01T00:00:00")] ForwardOrdered Irregular Points
units: %
name: cfc_night
Total size: 93.75 KB


In [4]:
using DimensionalData

In [5]:
lats=lookup(cfc_day,:lat)
lons=lookup(cfc_day,:lon)

Sampled{Float64} ForwardOrdered Regular Points
wrapping: -59.75:0.5:59.75

In [6]:
using GeoMakie, GLMakie

In [7]:
using GeoJSON
worldCountries = GeoJSON.read("../ne_110m_admin_0_countries.geojson")

FeatureCollection with 177 Features

In [69]:
fig=Figure(backgroundcolor=:black,resolution=(800,600),fontcolor=:white)
Label(fig[1,1:2],text="If you want sun in Europe in February,\ngo to the end of the earth (well almost)", color=:white,fontsize=30)
Label(fig[2,1:2],text="Based on CM SAF CLAAS February 2023", color=:white,fontsize=20)
ax=GeoAxis(fig[3,1],latlims=(30.25,66.75),lonlims=(-59.75,59.75),backgroundcolor=:white)
co=contourf!(ax,lons,lats,cfc_day.data[:,:,1],levels=0:5:100,colormap=cgrad(rev=true,:batlow))
contour!(ax,lons,lats,cfc_day.data[:,:,1],levels=0:5:100,colormap=cgrad(rev=true,:grays))

Colorbar(fig[3,2],co,vertical=true,label="Mean cloud coverage at daytime [%]",tickcolor=:white,labelcolor=:white,ticklabelcolor=:white)
hm1=poly!(
    ax, worldCountries;
    color= :transparent,
    strokecolor = :black,
    strokewidth = 1)
translate!(hm1, 0, 0, 1000)
hidespines!(ax)
hidedecorations!(ax)
ax=GeoAxis(fig[4,1],latlims=(30.25,66.75),lonlims=(-59.75,59.75),backgroundcolor=:black)
co=contourf!(ax,lons,lats,cfc_night.data[:,:,1],levels=0:5:100,colormap=cgrad(rev=true,:batlow))
contour!(ax,lons,lats,cfc_night.data[:,:,1],levels=0:5:100,colormap=cgrad(rev=true,:grays))
Colorbar(fig[4,2],co,vertical=true,label="Mean cloud coverage at night [%]",tickcolor=:white,labelcolor=:white,ticklabelcolor=:white)

hm1=poly!(
    ax, worldCountries;
    color= :transparent,
    strokecolor = :white,
    strokewidth = 1)
translate!(hm1, 0, 0, 1000)
hidespines!(ax)
hidedecorations!(ax)
Label(fig[5,1:2],text="Data: © Deutscher Wetterdienst\nMade with Natural Earth",justification=:left,color=:white)
#Makie.resize_to_layout!(fig)
fig


In [70]:
save("europe.png",fig)